- 참고 노트북 
    - https://excelsior-cjh.tistory.com/154?category=940399

In [2]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("../data", one_hot=True)

In [4]:
import sys
import tensorflow as tf
import numpy as np
mnist = tf.keras.datasets.mnist

In [5]:
mnist

<module 'tensorflow._api.v1.keras.datasets.mnist' from 'C:\\Users\\user\\Anaconda3\\lib\\site-packages\\tensorflow\\_api\\v1\\keras\\datasets\\mnist\\__init__.py'>

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# RNN 로우레벨 구현

In [7]:

#####################
# Define Parameters #
#####################
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

# Where to save TensorBoard model summaries
LOG_DIR = "./logs/RNN_with_summaries"

# MNIST 데이터 불러오기 위한 함수 정의
def mnist_load():
    (train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

    # Train set
    train_x = train_x.astype('float32') / 255.
    train_y = tf.keras.utils.to_categorical(train_y, num_classes=10)
    # Test set
    test_x = test_x.astype('float32') / 255.
    test_y = tf.keras.utils.to_categorical(test_y, num_classes=10)
    return (train_x, train_y), (test_x, test_y)

# MNIST 데이터 불러오기
(train_x, train_y), (test_x, test_y) = mnist_load()
dataset = tf.data.Dataset.from_tensor_slices(({"image": train_x}, train_y))
dataset = dataset.shuffle(100000).repeat().batch(batch_size)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()


In [8]:

# Create placeholders for inputs, labels
_inputs = tf.placeholder(tf.float32, 
                       shape=[None, time_steps, element_size], 
                       name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='labels')


In [9]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [10]:
 
  # Weights and bias for input and hidden layer
with tf.name_scope('rnn_weights'):
    with tf.name_scope('W_x'):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope('W_h'):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope('Bias'):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)


Instructions for updating:
Colocations handled automatically by placer.


In [11]:
def rnn_step(previous_hidden_state, x):
    current_hidden_state = tf.tanh(
        tf.matmul(previous_hidden_state, Wh) + 
        tf.matmul(x, Wx) + b_rnn)
    return current_hidden_state

In [12]:

# tf.scan() 함수로 입력값 처리
# input shape: (batch_size, time_steps, element_size)
processed_input = tf.transpose(_inputs, perm=[1, 0, 2])
# transposed input shape: (time_steps, batch_size, element_size)

initial_hidden = tf.zeros([batch_size, hidden_layer_size])
# time_steps에 따른 상태 벡터(state vector) 구하기
all_hidden_states = tf.scan(rnn_step, processed_input,
                          initializer=initial_hidden, name='states')


In [13]:

# tf.scan() 예제
import numpy as np
import tensorflow as tf
elems = np.array(['T', 'e', 'n', 's', 'o', 'r', ' ', 'F', 'l', 'o', 'w'])
scan_sum = tf.scan(lambda a, x: a + x, elems)
sess = tf.InteractiveSession()
sess.run(scan_sum)
# sess.close()


array([b'T', b'Te', b'Ten', b'Tens', b'Tenso', b'Tensor', b'Tensor ',
       b'Tensor F', b'Tensor Fl', b'Tensor Flo', b'Tensor Flow'],
      dtype=object)

In [14]:

# 출력에 적용할 가중치
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope('W_linear'):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], 
                                             mean=0, stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope('Bias_linear'):
        bl = tf.Variable(tf.truncated_normal([num_classes], 
                                             mean=0, stddev=.01))
        variable_summaries(bl)

# 상태 벡터에 linear layer 적용
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bl

with tf.name_scope('linear_layer_weights') as scope:
    # 시간에 따라 반복하면서 모든 RNN 결과에 get_linear_layer 적용
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    # 최종 결과 = h_28
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)


In [15]:

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y))
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    # RMSPropOptimizer 사용
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(output, 1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar('accuracy', accuracy)

# 요약을 병합
merged = tf.summary.merge_all()


In [16]:
 
# Get a small test set
test_data = test_x[:batch_size].reshape([-1, time_steps, element_size])
test_label = test_y[:batch_size]

with tf.Session() as sess:
    # LOG_DIR에 텐세보드에서 사용할 요약을 기록
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train', 
                                         graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test', 
                                        graph=tf.get_default_graph())
    sess.run(tf.global_variables_initializer())
      
          
    for step in range(10000):
        batch_x, batch_y = sess.run(next_batch)
        summary, _ = sess.run([merged, train_step], 
                              feed_dict={_inputs:batch_x['image'], y:batch_y})   
        # 요약 추가
        train_writer.add_summary(summary, step)
        
        if (step+1) % 1000 == 0:
            acc, loss = sess.run([accuracy, cross_entropy], 
                                 feed_dict={_inputs:batch_x['image'], y:batch_y})
            print("Iter: %04d\t" % (step+1), 
                  "MiniBatch Loss: {:.6f}\t".format(loss),
                  "Training Acc={:.5f}".format(acc))
            
        if (step+1) % 100 == 0:
            # MNIST 테스트 이미지에서 정확도를 계산해 요약에 추가
            summary, acc = sess.run([merged, accuracy], 
                                    feed_dict={_inputs: test_data, 
                                               y: test_label})
            test_writer.add_summary(summary, step)
                  
    test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,
                                             y: test_label})
    print("Test Accuracy:", test_acc)


Iter: 1000	 MiniBatch Loss: 1.312871	 Training Acc=49.21875
Iter: 2000	 MiniBatch Loss: 0.927880	 Training Acc=69.53125
Iter: 3000	 MiniBatch Loss: 0.296155	 Training Acc=92.18750
Iter: 4000	 MiniBatch Loss: 0.160103	 Training Acc=96.09375
Iter: 5000	 MiniBatch Loss: 0.149526	 Training Acc=96.09375
Iter: 6000	 MiniBatch Loss: 0.091280	 Training Acc=96.09375
Iter: 7000	 MiniBatch Loss: 0.036761	 Training Acc=100.00000
Iter: 8000	 MiniBatch Loss: 0.069959	 Training Acc=97.65625
Iter: 9000	 MiniBatch Loss: 0.010371	 Training Acc=100.00000
Iter: 10000	 MiniBatch Loss: 0.047497	 Training Acc=98.43750
Test Accuracy: 96.09375


# 텐서플로 내장 RNN

In [18]:
# Parameters
element_size = 28 
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

In [19]:
# 1) Create placeholders for inputs, labels
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps,
                                          element_size], name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='inputs')

In [20]:
# 2) RNN Model
# Tensorflow built-in functions
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, _inputs, dtype=tf.float32)

Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],
                                   mean=0, stddev=.01))
bl = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=.01))

def get_linear_layer(vector):
    return tf.matmul(vector, Wl) + bl

last_rnn_output = outputs[:,-1,:]
final_output = get_linear_layer(last_rnn_output)


Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [21]:
# 3) Loss function
cross_entropy = tf.reduce_mean(
                  tf.nn.softmax_cross_entropy_with_logits_v2(logits=final_output, labels=y))
# 4) Optimizer
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

# 5) accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(final_output, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100


In [ ]:
# 6) Training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Get a small test set
    test_data = test_x[:batch_size].reshape([-1, time_steps, element_size])
    test_label = test_y[:batch_size]

    for step in range(10000):
        batch_x, batch_y = sess.run(next_batch)

        sess.run(train_step, feed_dict={_inputs:batch_x['image'], 
                                        y:batch_y})          


        if (step+1) % 1000 == 0:
            acc, loss = sess.run([accuracy, cross_entropy], 
                                 feed_dict={_inputs:batch_x['image'], y:batch_y})
            print("Iter: %04d\t" % (step+1), 
                  "MiniBatch Loss: {:.6f}\t".format(loss),
                  "Training Acc={:.5f}".format(acc))

        if (step+1) % 100 == 0:
            # MNIST 테스트 이미지에서 정확도를 계산해 요약에 추가
            acc = sess.run(accuracy, feed_dict={_inputs: test_data, 
                                                y: test_label})
            
    test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,
                                             y: test_label})
    print("Test Accuracy:", test_acc)


Iter: 1000	 MiniBatch Loss: 0.169439	 Training Acc=95.31250
Iter: 2000	 MiniBatch Loss: 0.111478	 Training Acc=96.09375
Iter: 3000	 MiniBatch Loss: 0.057166	 Training Acc=99.21875
Iter: 4000	 MiniBatch Loss: 0.012139	 Training Acc=100.00000
Iter: 5000	 MiniBatch Loss: 0.071971	 Training Acc=98.43750
Iter: 6000	 MiniBatch Loss: 0.029778	 Training Acc=99.21875
Iter: 7000	 MiniBatch Loss: 0.067311	 Training Acc=99.21875
Iter: 8000	 MiniBatch Loss: 0.088630	 Training Acc=96.87500
